# **Train Siamese**

In [ ]:
%cd /content/drive/MyDrive/

/content/drive/MyDrive


In [ ]:
import numpy as np
import pandas as pd
import random
train_summer = np.load("./place_recognition/summer_train_150.npy")
train_fall = np.load("./place_recognition/fall_train_150.npy")
print("summer image shape is ", train_summer.shape)
print("fall image shape is ", train_fall.shape)

summer image shape is  (3000, 150, 150, 3)
fall image shape is  (3000, 150, 150, 3)


In [ ]:
train_summer = train_summer / 255.0
train_fall = train_fall / 255.0

In [ ]:
import keras
from keras import Model
from keras import models
from keras.layers import *
from keras.optimizers import RMSprop, Adam
import tensorflow as tf

In [ ]:
vgg = tf.keras.applications.VGG16(include_top=False, weights='imagenet', input_shape=(150,150,3))
vgg.trainable = False

In [ ]:
from tensorflow.keras.layers import Input, Dropout, Dense, Flatten, BatchNormalization, Activation, MaxPooling2D
from tensorflow.keras.models import Model
import tensorflow.keras.backend as K

def build_siamese_model(inputShape):
  inputs = Input(inputShape)
  x = vgg(inputs)
  x = Dense(4096, activation='relu')(x)
  x = Dropout(0.3)(x)
  x = Dense(2048, activation='relu')(x)
  x = Dropout(0.3)(x)
  x = Flatten()(x)
  outputs = Dense(128)(x)
  model = Model(inputs,outputs)
  #model.summary()
  return model

def euclidean_distance(vectors):
  (featsA, featsB) = vectors

  sumSquared = K.sum(K.square(featsA- featsB), axis=1, keepdims=True)

  return K.sqrt(K.maximum(sumSquared, K.epsilon()))

def make_pairs(summer, fall, Mode=0):

  pairImages = []
  pairLabels = []

  numPlace = np.arange(len(summer))

  for idx in numPlace:
    currentImage = summer[idx]
    posImage = fall[idx]
    
    pairImages.append([currentImage,posImage])
    pairLabels.append([1])

  if Mode == 1:
    for idx in numPlace:
      currentImage = summer[idx]

      for num in range(4):
        negIdx = np.where(numPlace != idx)[0]
        negImage = fall[np.random.choice(negIdx)]
        pairImages.append([currentImage,negImage])
        pairLabels.append([0])

  return (np.array(pairImages), np.array(pairLabels))

In [ ]:
from tensorflow.keras.layers import Lambda,Input, Dropout, Dense, Flatten
from tensorflow.keras.models import Model
imgA = Input(shape=(150,150,3))
imgB = Input(shape=(150,150,3))
#descriptorExtractor라는 모델 객체를 만든다.
descriptorExtractor = build_siamese_model((150,150,3))
#이제 해당 모델의 출력으로 각 global descriptor를 출력한다.
descsA = descriptorExtractor(imgA)
descsB = descriptorExtractor(imgB)

#람다(lamda)레이어를 통해서, descriptor 사이의 유클리드 거리를 구한다.
distance = Lambda(euclidean_distance)([descsA,descsB])
#이제 출력을 sigmoid를 사용하여 0과 1사이의 출력이 되도록한다.
outputs = Dense(1, activation='sigmoid')(distance)

#최종적으로 입력과 출력을 등록하여 모델 객체를 생성한다.
final_model = Model(inputs=[imgA, imgB], outputs=outputs)
final_model.summary()

In [ ]:
train_summer = train_summer[:250]
train_fall = train_fall[:250]

In [ ]:
(negimg, neglabel) = make_pairs(train_summer, train_fall, 1)

In [ ]:
sum = negimg[:,0,:,:,:]
fal = negimg[:,1,:,:,:]
final_model.compile(optimizer='adam', loss='mse', metrics=["accuracy"] )
final_model.fit([sum,fal], neglabel, batch_size=100, epochs=100)

In [ ]:
from keras.models import save_model, load_model

#final_model.save('./place_recognition/cnn_model2.h5')

In [ ]:
descsA = descriptorExtractor.predict(train_summer)
descsB = descriptorExtractor.predict(train_fall)

In [ ]:
def cosine_distance(a, b):
    return abs(np.dot(a, b) / (np.linalg.norm(a) * (np.linalg.norm(b))))
    
def similarity_matrix(a,b):
  w_len, h_len = a.shape[0], b.shape[0]
  matrix = np.zeros((w_len, h_len))

  for i in range(w_len):
    for j in range(h_len):
      matrix[i][j] = cosine_distance(a[i],b[j])

  return matrix

def scaling(matrix):
  result = (matrix -np.min(matrix))/(np.max(matrix) - np.min(matrix))
  
  return result

In [ ]:
import seaborn as sns
summer_feature = descsA[:250]
fall_feature = descsB[:250]


matrix = similarity_matrix(summer_feature, fall_feature)
matrix = scaling(matrix)
sns.heatmap(matrix)

In [ ]:
# Precision-Recall function
import sklearn.metrics as metrics
import matplotlib.pyplot as plt
def Precision_Recall(similarity):
  thresholds = [i / 1000 for i in range(1000)]
  matrix_shape = len(matrix)
  GT = np.eye(matrix_shape)
  pr_s = []
  re_s = []
  f1_s = []

  for threshold in thresholds:
    test_matrix = similarity > threshold

    right = np.sum(test_matrix * GT == 1)
    precision = right / np.sum(test_matrix)
    recall = right/ np.sum(GT)
    f1 = 2 * precision*recall/(precision+recall)

    pr_s.append(precision)
    re_s.append(recall)
    f1_s.append(f1)

  return pr_s, re_s, f1_s

In [ ]:
Precision, Recall, F1 = Precision_Recall(matrix)

plt.plot(Recall,Precision,'-r',label="CAE")
plt.xlabel("Recall")
plt.ylabel("Precision")
plt.title("Recall-Precision curve")
plt.legend(loc='upper right')
plt.grid()
plt.show()

In [ ]:
# f1_score_function
f1_max = max(F1)
print("f1_max = ", round(f1_max,4))

# **Test_Siamese**

In [ ]:
%cd /content/drive/MyDrive/

In [ ]:
import numpy as np
test_summer = np.load("./place_recognition/test_summer.npy")
test_fall = np.load("./place_recognition/test_fall.npy")
print("summer image shape is ", test_summer.shape)
print("fall image shape is ", test_fall.shape)

In [ ]:
test_summer = test_summer / 255.0
test_fall = test_fall / 255.0

In [ ]:
from keras.models import load_model
recent_model = load_model('./place_recognition/cnn_model2.h5')

In [ ]:
summer_feature = recent_model.get_layer('model').predict(test_summer)
fall_feature = recent_model.get_layer('model').predict(test_fall)

print("summer_feature size = ", summer_feature.shape)
print("fall_feature size = ", fall_feature.shape)

In [ ]:
def cosine_distance(a, b):
    return abs(np.dot(a, b) / (np.linalg.norm(a) * (np.linalg.norm(b))))
    
def similarity_matrix(a,b):
  w_len, h_len = a.shape[0], b.shape[0]
  matrix = np.zeros((w_len, h_len))

  for i in range(w_len):
    for j in range(h_len):
      matrix[i][j] = cosine_distance(a[i],b[j])

  return matrix

def scaling(matrix):
  result = (matrix -np.min(matrix))/(np.max(matrix) - np.min(matrix))
  
  return result

In [ ]:
import seaborn as sns
matrix = similarity_matrix(summer_feature, fall_feature)
matrix = scaling(matrix)
np.save('similarity_matrix.npy',matrix)
sns.heatmap(matrix)

In [ ]:
# Precision-Recall function
import sklearn.metrics as metrics
import matplotlib.pyplot as plt

def Precision_Recall(similarity):
  thresholds = [i / 1000 for i in range(1000)]
  matrix_shape = len(matrix)
  GT = np.eye(matrix_shape)
  pr_s = []
  re_s = []
  f1_s = []

  for threshold in thresholds:
    test_matrix = similarity > threshold

    right = np.sum(test_matrix * GT == 1)
    precision = right / np.sum(test_matrix)
    recall = right/ np.sum(GT)
    f1 = 2 * precision*recall/(precision+recall)

    pr_s.append(precision)
    re_s.append(recall)
    f1_s.append(f1)

  return pr_s, re_s, f1_s

In [ ]:
Precision, Recall, F1 = Precision_Recall(matrix)

plt.plot(Recall,Precision,'-r',label="CAE")
plt.xlabel("Recall")
plt.ylabel("Precision")
plt.title("Recall-Precision curve")
plt.legend(loc='upper right')
plt.grid()
plt.show()

In [ ]:
# f1_score_function
f1_max = max(F1)
print("f1_max = ", round(f1_max,4))

# **Train CAE**

In [ ]:
%cd /content/drive/MyDrive/

In [ ]:
import numpy as np
import tensorflow as tf
import cv2
import skimage.transform
train_summer = np.load("./place_recognition/summer_train_150.npy")
train_fall = np.load("./place_recognition/fall_train_150.npy")
print("summer image shape is ", train_summer.shape)
print("fall image shape is ", train_fall.shape)

In [ ]:
train_summer = np.transpose(train_summer, (1, 2, 3, 0))
train_summer = skimage.transform.resize(train_summer, (128, 128, 3, 3000)) 
train_summer = np.transpose(train_summer, (3, 0, 1, 2) ) # move back 

train_fall = np.transpose(train_fall, (1, 2, 3, 0))
train_fall = skimage.transform.resize(train_fall, (128, 128, 3, 3000)) 
train_fall = np.transpose(train_fall, (3, 0, 1, 2) ) # move back 

print(train_summer.shape,'\n',train_fall.shape)

In [ ]:
vgg = tf.keras.applications.VGG16(include_top=False, weights='imagenet', input_shape=(128,128,3))
vgg.trainable = False

In [ ]:
from tensorflow.keras.layers import Input, Dense, Conv2D, MaxPooling2D, Flatten, UpSampling2D, Reshape
from tensorflow.keras.models import Sequential, Model

inputs = Input((128,128,3))
x = vgg(inputs)
x = Flatten()(x)
encoded = Dense(300, activation='relu')(x)

encoder = Model(inputs, encoded)
#encoder.summary()

Decoded_img = Input(shape=(300,))
x = Dense(4*4*512)(Decoded_img)
x = Reshape((4,4,512))(x)

x = UpSampling2D((2, 2))(x)
x = Conv2D(512, (3, 3), activation='relu', padding='same')(x)
x = Conv2D(512, (3, 3), activation='relu', padding='same')(x)
x = Conv2D(512, (3, 3), activation='relu', padding='same')(x)

x = UpSampling2D((2, 2))(x)
x = Conv2D(512, (3, 3), activation='relu', padding='same')(x)
x = Conv2D(512, (3, 3), activation='relu', padding='same')(x)
x = Conv2D(512, (3, 3), activation='relu', padding='same')(x)

x = UpSampling2D((2, 2))(x)
x = Conv2D(256, (3, 3), activation='relu', padding='same')(x)
x = Conv2D(256, (3, 3), activation='relu', padding='same')(x)
x = Conv2D(256, (3, 3), activation='relu', padding='same')(x)

x = UpSampling2D((2, 2))(x)
x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)

x = UpSampling2D((2, 2))(x)
x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)

decoded = Conv2D(3, (3, 3), activation='sigmoid',padding='same')(x)
decoder = Model(Decoded_img, decoded)
#decoder.summary()

x = encoder(inputs)
decoder_out = decoder(x)
autoencoder = Model(inputs, decoder_out)
#autoencoder.summary()
autoencoder.compile(optimizer='adam', loss='mse', metrics=["accuracy"] )

In [ ]:
autoencoder.fit(train_summer, train_fall, batch_size = 100, epochs= 50)

In [ ]:
summer_feature = encoder.predict(train_summer)
fall_feature = encoder.predict(train_fall)

print("summer_feature size = ", summer_feature.shape)
print("fall_feature size = ", fall_feature.shape)

In [ ]:
def cosine_distance(a, b):
    return abs(np.dot(a, b) / (np.linalg.norm(a) * (np.linalg.norm(b))))
    
def similarity_matrix(a,b):
  w_len, h_len = a.shape[0], b.shape[0]
  matrix = np.zeros((w_len, h_len))

  for i in range(w_len):
    for j in range(h_len):
      matrix[i][j] = cosine_distance(a[i],b[j])

  return matrix

def scaling(matrix):
  result = (matrix -np.min(matrix))/(np.max(matrix) - np.min(matrix))
  
  return result

In [ ]:
import seaborn as sns
summer_feature = summer_feature[:250]
fall_feature = fall_feature[:250]


matrix = similarity_matrix(summer_feature, fall_feature)
matrix = scaling(matrix)

sns.heatmap(matrix)

In [ ]:
# Precision-Recall function
import sklearn.metrics as metrics
import matplotlib.pyplot as plt

def Precision_Recall(similarity):
  thresholds = [i / 1000 for i in range(1000)]
  matrix_shape = len(matrix)
  GT = np.eye(matrix_shape)
  pr_s = []
  re_s = []
  f1_s = []

  for threshold in thresholds:
    test_matrix = similarity > threshold

    right = np.sum(test_matrix * GT == 1)
    precision = right / np.sum(test_matrix)
    recall = right/ np.sum(GT)
    f1 = 2 * precision*recall/(precision+recall)

    pr_s.append(precision)
    re_s.append(recall)
    f1_s.append(f1)

  return pr_s, re_s, f1_s

In [ ]:
Precision, Recall, F1 = Precision_Recall(matrix)

plt.plot(Recall,Precision,'-r',label="CAE")
plt.xlabel("Recall")
plt.ylabel("Precision")
plt.title("Recall-Precision curve")
plt.legend(loc='upper right')
plt.grid()
plt.show()

In [ ]:
# f1_score_function
f1_max = max(F1)
print("f1_max = ", round(f1_max,4))

# **Test_CAE**

In [ ]:
import numpy as np
import tensorflow as tf
import cv2
import skimage.transform
test_summer = np.load("./place_recognition/test_summer.npy")
test_fall = np.load("./place_recognition/test_fall.npy")
print("summer image shape is ", test_summer.shape)
print("fall image shape is ", test_fall.shape)

summer image shape is  (250, 150, 150, 3)
fall image shape is  (250, 150, 150, 3)


In [ ]:
test_summer = np.transpose(test_summer, (1, 2, 3, 0))
test_summer = skimage.transform.resize(test_summer, (128, 128, 3, 250)) 
test_summer = np.transpose(test_summer, (3, 0, 1, 2) ) # move back 

test_fall = np.transpose(test_fall, (1, 2, 3, 0))
test_fall = skimage.transform.resize(test_fall, (128, 128, 3, 250)) 
test_fall = np.transpose(test_fall, (3, 0, 1, 2) ) # move back 

print(test_summer.shape,'\n',test_fall.shape)

(250, 128, 128, 3) 
 (250, 128, 128, 3)


In [ ]:
from keras.models import load_model
recent_model = load_model('./place_recognition/CAE_model.h5')

In [ ]:
summer_feature = recent_model.get_layer('model').predict(test_summer)
fall_feature = recent_model.get_layer('model').predict(test_fall)

print("summer_feature size = ", summer_feature.shape)
print("fall_feature size = ", fall_feature.shape)

summer_feature size =  (250, 300)
fall_feature size =  (250, 300)


In [ ]:
def cosine_distance(a, b):
    return abs(np.dot(a, b) / (np.linalg.norm(a) * (np.linalg.norm(b))))
    
def similarity_matrix(a,b):
  w_len, h_len = a.shape[0], b.shape[0]
  matrix = np.zeros((w_len, h_len))

  for i in range(w_len):
    for j in range(h_len):
      matrix[i][j] = cosine_distance(a[i],b[j])

  return matrix

def scaling(matrix):
  result = (matrix -np.min(matrix))/(np.max(matrix) - np.min(matrix))
  
  return result

In [ ]:
import seaborn as sns
matrix = similarity_matrix(summer_feature, fall_feature)
matrix = scaling(matrix)

sns.heatmap(matrix)

In [ ]:
# Precision-Recall function
import sklearn.metrics as metrics
import matplotlib.pyplot as plt

def Precision_Recall(similarity):
  thresholds = [i / 1000 for i in range(1000)]
  matrix_shape = len(matrix)
  GT = np.eye(matrix_shape)
  pr_s = []
  re_s = []
  f1_s = []

  for threshold in thresholds:
    test_matrix = similarity > threshold

    right = np.sum(test_matrix * GT == 1)
    precision = right / np.sum(test_matrix)
    recall = right/ np.sum(GT)
    f1 = 2 * precision*recall/(precision+recall)

    pr_s.append(precision)
    re_s.append(recall)
    f1_s.append(f1)

  return pr_s, re_s, f1_s

In [ ]:
Precision, Recall, F1 = Precision_Recall(matrix)

plt.plot(Recall,Precision,'-r',label="CAE")
plt.xlabel("Recall")
plt.ylabel("Precision")
plt.title("Recall-Precision curve")
plt.legend(loc='upper right')
plt.grid()
plt.show()

In [ ]:
# f1_score_function
f1_max = max(F1)
print("f1_max = ", round(f1_max,4))